In [ ]:
import os
import numpy as np
import glob
from pathlib import Path
import re

for path in Path('src').rglob('*.c'):
    print(path.name)

# for dataset in ['Bike_Sharing_Demand']:
for dataset in ['abalone', 'house_sales', 'analcatdata_supreme', 'Bike_Sharing_Demand']:
    before_mse_list, after_mse_list = [], []
    log_dir = f"log/regression_dem/{dataset}"

    before_p = re.compile('^test_loss before adaptation')
    after_p = re.compile('^test_loss after adaptation')

    for path in Path(log_dir).rglob('*.txt'):
        f = "".join(open(path, "r").readlines())
        before_mse = before_p.search(f)
        after_mse = after_p.search(f)

        before_mse_list.append(before_mse)
        after_mse_list.append(after_mse)
    before_mse_list = np.array(before_mse_list)
    after_mse_list = np.array(after_mse_list)

    before_mean = np.mean(before_mse_list)
    before_se = np.std(before_mse_list, ddof=1) / np.sqrt(len(before_mse_list))

    after_mean = np.mean(after_mse_list)
    after_se = np.std(after_mse_list, ddof=1) / np.sqrt(len(after_mse_list))

    print(f"dataset: {dataset}")
    print(f"before: {before_mean:.4f} ± {before_se:.4f}")
    print(f"after: {after_mean:.4f} ± {after_se:.4f}")